# Get one symbol nakeds

 - [x] get equity fno list
 - [x] get equity histories
 - [x] get index histories
 - [ ] get equity option histories
 - [ ] get index option histories
 - [ ] get lot-sizes
 - [ ] get market price and volatility
 - [ ] get chains
 - [ ] get rim SDs
 - [ ] retest SDs
 - [ ] get margins
 - [ ] pack into symbol objects



In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

from loguru import logger

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_async import util
util.startLoop()

# Set the root
from from_root import from_root
ROOT = from_root()

logger.add(sink=ROOT/ 'log' / 'zscrap.log', mode='w')

In [ ]:

# imports
import json
from datetime import datetime, timedelta, timezone

import requests
from tqdm import tqdm

# NSELive - Jugaad's clone

In [ ]:
def live_cache(app_name):
    """Caches the output for time_out specified. This is done in order to
    prevent hitting live quote requests to NSE too frequently. This wrapper
    will fetch the quote/live result first time and return the same result for
    any calls within 'time_out' seconds.

    Logic:
        key = concat of args
        try:
            cached_value = self._cache[key]
            if now - self._cache['tstamp'] < time_out
                return cached_value['value']
        except AttributeError: # _cache attribute has not been created yet
            self._cache = {}
        finally:
            val = fetch-new-value
            new_value = {'tstamp': now, 'value': val}
            self._cache[key] = new_value
            return val

    """
    def wrapper(self, *args, **kwargs):
        """Wrapper function which calls the function only after the timeout,
        otherwise returns value from the cache.

        """
        # Get key by just concating the list of args and kwargs values and hope
        # that it does not break the code :P 
        inputs =  [str(a) for a in args] + [str(kwargs[k]) for k in kwargs]
        key = app_name.__name__ + '-'.join(inputs)
        now = datetime.now()
        time_out = self.time_out
        try:
            cache_obj = self._cache[key]
            if now - cache_obj['timestamp'] < timedelta(seconds=time_out):
                return cache_obj['value']
        except:
            self._cache = {}
        value = app_name(self, *args, **kwargs)
        self._cache[key] = {'value': value, 'timestamp': now}
        return value

    return wrapper 

In [ ]:

class NSELive:
    time_out = 5
    base_url = "https://www.nseindia.com/api"
    page_url = "https://www.nseindia.com/get-quotes/equity?symbol=LT"
    _routes = {
            "stock_meta": "/equity-meta-info",
            "stock_quote": "/quote-equity",
            "stock_derivative_quote": "/quote-derivative",
            "market_status": "/marketStatus",
            "chart_data": "/chart-databyindex",
            "market_turnover": "/market-turnover",
            "equity_derivative_turnover": "/equity-stock",
            "all_indices": "/allIndices",
            "live_index": "/equity-stockIndices",
            "index_option_chain": "/option-chain-indices",
            "equity_option_chain": "/option-chain-equities",
            "currency_option_chain": "/option-chain-currency",
            "pre_open_market": "/market-data-pre-open",
            "holiday_list": "/holiday-master?type=trading",
            "stock_history" : "/historical/cm/equity", # added by rkv
    }
    
    def __init__(self):
        self.s = requests.Session()
        h = {
            "Host": "www.nseindia.com",
            "Referer": "https://www.nseindia.com/get-quotes/equity?symbol=SBIN",
            "X-Requested-With": "XMLHttpRequest",
            "pragma": "no-cache",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
            }
        self.s.headers.update(h)
        self.s.get(self.page_url)

    def split_dates(self, symbol, days:int=365, chunks:int=50) -> list:
        end = datetime.today()
        periods = int(days/chunks)
        start = end-timedelta(days=days)

        if days < chunks:
            date_ranges = [(start, end)]
        else:   
            dates = pd.date_range(start, end, periods).date
            date_ranges = list(zip(pd.Series(dates), 
                                   pd.Series(dates).shift(-1)+timedelta(days=-1)))[:-1]

        # remove last tuple having period as NaT
        if any(pd.isna(e) for element in date_ranges for e in element):
            date_ranges = date_ranges[:-1]

        series = 'EQ'

        ranges = [{'symbol': symbol,
        'from': start.strftime('%d-%m-%Y'), 
        'to': end.strftime('%d-%m-%Y'),
        'series': f'["{series}"]'} for start, end in date_ranges]

        return ranges 

    def get(self, route, payload={}):
        url = self.base_url + self._routes[route]
        r = self.s.get(url, params=payload)
        return r.json()

    @live_cache
    def stock_quote(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_quote", data) 

    @live_cache
    def stock_quote_fno(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_derivative_quote", data)

    @live_cache
    def trade_info(self, symbol):
        data = {"symbol": symbol, "section": "trade_info"}
        return self.get("stock_quote", data) 

    @live_cache
    def market_status(self):
        return self.get("market_status", {})

    @live_cache
    def chart_data(self, symbol, indices=False):
        data = {"index" : symbol + "EQN"}
        if indices:
            data["index"] = symbol
            data["indices"] = "true"
        return self.get("chart_data", data)
    
    @live_cache
    def tick_data(self, symbol, indices=False):
        return self.chart_data(symbol, indices)

    @live_cache
    def market_turnover(self):
        return self.get("market_turnover")

    @live_cache
    def eq_derivative_turnover(self, type="allcontracts"):
        data = {"index": type}
        return self.get("equity_derivative_turnover", data)
    
    @live_cache
    def all_indices(self):
        return self.get("all_indices")

    def live_index(self, symbol="NIFTY 50"):
        data = {"index" : symbol}
        return self.get("live_index", data)
    
    @live_cache
    def index_option_chain(self, symbol="NIFTY"):
        data = {"symbol": symbol}
        return self.get("index_option_chain", data)

    @live_cache
    def equities_option_chain(self, symbol):
        data = {"symbol": symbol}
        return self.get("equity_option_chain", data)

    @live_cache
    def currency_option_chain(self, symbol="USDINR"):
        data = {"symbol": symbol}
        return self.get("currency_option_chain", data)

    @live_cache
    def live_fno(self):
        return self.live_index("SECURITIES IN F&O")
    
    @live_cache
    def pre_open_market(self, key="NIFTY"):
        data = {"key": key}
        return self.get("pre_open_market", data)
    
    @live_cache
    def holiday_list(self):
        return self.get("holiday_list", {})
    
    @live_cache
    def stock_history(self, symbol, days:int=365, chunks:int=50):
        date_ranges = self.split_dates(symbol=symbol, 
                                       days=days, 
                                       chunks=chunks)
        
        result = []
        for dr in date_ranges:
            result.append(self.get("stock_history", dr))

        df = pd.concat([pd.DataFrame(r.get('data')) for r in result], axis=0, ignore_index=True)


        # ...clean columns

        mapping = {'CH_SYMBOL': 'nse_symbol',
        'TIMESTAMP': 'date',
        'CH_OPENING_PRICE': 'open',
        'CH_TRADE_HIGH_PRICE': 'high',
        'CH_TRADE_LOW_PRICE': 'low',
        'CH_CLOSING_PRICE': 'close',
        'CH_TOT_TRADED_QTY':'qty_traded',
        'CH_TOT_TRADED_VAL':'value_traded',
        'CH_TOTAL_TRADES': 'trades',
        'VWAP': 'vwap',
        'updatedAt': 'extracted_on',
        }

        df = df[[col for col in mapping.keys() if col in df.columns]].rename(columns=mapping)

        # ...convert column datatypes

        astype_map = {**{k: 'float' for k in 
                        ['open', 'high', 'low', 'close', 
                        'value_traded', 'trades', 'vwap']},
                        **{'qty_traded': 'int'}}

        df = df.astype(astype_map)

        # ...change date columns to utc

        replace_cols = ['date', 'extracted_on']
        df1 = df[replace_cols].map(lambda x: datetime.fromisoformat(x))
        df = df.assign(date=df1.date, extracted_on=df1.extracted_on)

        return df


## Initialize NSELive

In [ ]:
nse = NSELive()

## Get Equity fno list

In [ ]:
equities = nse.live_fno()

# Equities list
fno_equities = {kv.get('symbol') for kv in equities.get('data')}

## Get Stock History

In [ ]:
df = nse.stock_history('ongc', days=365, chunks=50)

In [ ]:
df.head()

# Index History

In [ ]:
class IDXHistories:

    time_out = 5
    base_url = 'https://niftyindices.com'
    url = 'https://niftyindices.com/Backpage.aspx/getHistoricaldatatabletoString'

    # prepare `post` header
    post_header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/91.0.4472.77 Safari/537.36',   
            'Connection': 'keep-alive',
            'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'DNT': '1',
            'X-Requested-With': 'XMLHttpRequest',
            'sec-ch-ua-mobile': '?0',
            'Content-Type': 'application/json; charset=UTF-8',
            'Origin': 'https://niftyindices.com',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Dest': 'empty',
            'Referer': 'https://niftyindices.com/reports/historical-data',
            'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
        }

    def __init__(self, days:int=365) -> None:
        self.s = requests.Session()

        # update session with default headers and get the cookies
        init_header = requests.utils.default_headers()
        init_header.update(
            {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/91.0.4472.77 Safari/537.36', 
            }
        )
        self.s.headers.update(init_header)
        c = self.s.get(url=self.url)
        self.cookies = c.cookies

    def get(self, payload={}):

        r = self.s.post(url=self.url, headers=self.post_header,
                        cookies=self.cookies, data=payload, timeout=self.time_out)
        
        return r.json()

    def make_histories(self, days:int=365):
        """Makes histories for NIFTY50 and BANKNIFTY, based on number of days provided"""

        # build the date buckets and nse index symbols

        self.days = days # for the record

        end = datetime.today()
        chunks = 100
        periods = int(days/chunks)
        start = end-timedelta(days=days)

        if days < chunks:
            date_ranges = [(start, end)]
        else:   
            dates = pd.date_range(start, end, periods).date
            date_ranges = list(zip(pd.Series(dates), pd.Series(dates).shift(-1)+timedelta(days=-1)))[:-1]

        # remove last tuple having period as NaT
        if any(pd.isna(e) for element in date_ranges for e in element):
            date_ranges = date_ranges[:-1]

        idx_symbols =['Nifty Bank', 'Nifty 50']

        # organize the payloads
        payloads = [{'cinfo': str({'name': idx_symbol,
                                'startDate': s.strftime('%d-%b-%Y'),
                                'endDate': e.strftime('%d-%b-%Y'),
                                'indexName': idx_symbol})} for s, e in date_ranges for idx_symbol in idx_symbols]
        # get the raw jsons
        results = []

        for payload in tqdm(payloads):
            r = self.get(payload=json.dumps(payload))
            results.append(r)

        # build the dataframe
        df = pd.concat([pd.DataFrame(json.loads(r.get('d'))) for r in results],
                ignore_index=True)

        # clean the df

        #...drop unnecessary columns

        df = df.drop(df.columns[[0, 1]], axis=1)

        #...rename
        df.columns=['nse_symbol', 'date', 'open', 
                    'high', 'low', 'close']

        #...convert nse_symbol to IB's symbol
        df = pd.concat([df.nse_symbol.map({'Nifty Bank': 'BANKNIFTY', 
                        'Nifty 50': 'NIFTY50'}).rename('symbol'), 
                    df], axis=1)
        #...convert dates to UTC 3:30 PM IST (market close time)
        dates = df.date

        # .....specify desired timezone offset (Asia/Kolkata is UTC+5:30)
        tz_offset = timezone(timedelta(hours=5, minutes=30))  

        # .....parse dates with specified format and set time to 3:30 PM IST
        datetime_series = pd.to_datetime(dates, format='%d %b %Y') + pd.Timedelta(hours=15, minutes=30)

        # .....convert to UTC with desired offset
        utc_dates = datetime_series.dt.tz_localize(tz_offset)

        df = df.assign(date=utc_dates)

        # .....convert ohlc to numeric
        convert_dict = {k: 'float' for k in 
                        ['open', 'high', 'low', 'close']}

        df = df.astype(convert_dict)

        # .....sort by date
        df.sort_values(['nse_symbol', 'date'], inplace=True, ignore_index=True)

        # .....add extract_date
        now = datetime.now()
        utc_now = now.astimezone(timezone.utc)
        df = df.assign(extracted_on=utc_now)

        self.hist = df


## Initialize Index Histories

In [ ]:
idx = IDXHistories()

idx.make_histories() #give number of days needed

idx.hist

## convert nse to ib friendly symbols

In [ ]:
raw_fnos = nse.live_fno()
fno_list = {data.get('symbol') for data in raw_fnos.get('data')}

In [ ]:
subs = {"M&M": "MM",
"M&MFIN": "MMFIN",
"L&TFH": "LTFH",
"NIFTY": "NIFTY50"}

list_without_percent_sign = list(map(subs.get, fno_list, fno_list))

# fix length to 9 characters
ib_equity_fnos = [s[:9] for s in list_without_percent_sign]